In [ ]:
import time
import os
import numpy as np
from pynq import allocate
from pynq import MMIO
from pynq import Overlay

In [ ]:
os.getcwd()
ol = Overlay("spi_individual_bmi160.bit")

In [ ]:
# base addr
SPI_CONTROL = 0x40000000

# regs
soft_rst_n_reg    = 0x10
chip_reg          = 0x18
cpol_reg          = 0x20
cpha_reg          = 0x28
w_r_mode_reg      = 0x30
wr_width_reg      = 0x38
wr_data_reg       = 0x40
rd_width_reg      = 0x48
rd_target_num_reg = 0x50
wr_done_reg       = 0x58
rd_done_reg       = 0x68
rd_data_reg       = 0x78

# SPI从机列表(rd_mode=0-只读模式;rd_mode=1-指令控读模式)
chip_settings = {
    "MAX6675": {"chip_num":0, "cpol":0, "cpha":0, "wr_width":0, "rd_mode":0, "rd_comd_width":0, "rd_data_width":16},
    "BMI160":  {"chip_num":1, "cpol":1, "cpha":1, "wr_width":16, "rd_mode":1, "rd_comd_width":8, "rd_data_width":8},
}

In [ ]:
# 写寄存器
def write_reg(base_addr, offset, val):
    mmio = MMIO(base_addr, 64*1024)   # 创建一个名为mmio的MMIO对象,可用地址空间为64KB(与VIVADO设计中的地址分配相吻合)
    mmio.write(offset, val)           # 向地址为base_addr+offset的寄存器写入val


# 读寄存器
def read_reg(base_addr, offset):  
    mmio = MMIO(base_addr, 64*1024)
    read_val = mmio.read(offset)
    return read_val


# 写SPI
def spi_write(chip, wr_data):
    if chip in chip_settings:
        chip_num = chip_settings[chip]["chip_num"]
        cpol = chip_settings[chip]["cpol"]
        cpha = chip_settings[chip]["cpha"]
        wr_width = chip_settings[chip]["wr_width"]
    write_reg(SPI_CONTROL, soft_rst_n_reg, 0x0)
    write_reg(SPI_CONTROL, chip_reg, chip_num)
    write_reg(SPI_CONTROL, cpol_reg, cpol)
    write_reg(SPI_CONTROL, cpha_reg, cpha)
    write_reg(SPI_CONTROL, w_r_mode_reg, 0x1)
    write_reg(SPI_CONTROL, wr_width_reg, wr_width)
    write_reg(SPI_CONTROL, wr_data_reg, wr_data)
    write_reg(SPI_CONTROL, soft_rst_n_reg, 0x1)
    wr_done = read_reg(SPI_CONTROL, wr_done_reg)
    while wr_done!=1 :
        wr_done = read_reg(SPI_CONTROL, wr_done_reg)
    print("Successfully write.")


# 读SPI
def spi_read(chip, rd_command):
    if chip in chip_settings:
        chip_num = chip_settings[chip]["chip_num"]
        cpol = chip_settings[chip]["cpol"]
        cpha = chip_settings[chip]["cpha"]
        rd_mode = chip_settings[chip]["rd_mode"]
        rd_comd_width = chip_settings[chip]["rd_comd_width"]
        rd_data_width = chip_settings[chip]["rd_data_width"]
    if rd_mode==0 :
        write_reg(SPI_CONTROL, soft_rst_n_reg, 0x0)
        write_reg(SPI_CONTROL, chip_reg, chip_num)
        write_reg(SPI_CONTROL, cpol_reg, cpol)
        write_reg(SPI_CONTROL, cpha_reg, cpha)
        write_reg(SPI_CONTROL, w_r_mode_reg, 0x0)
        write_reg(SPI_CONTROL, rd_width_reg, rd_data_width)
        write_reg(SPI_CONTROL, rd_target_num_reg, 0x1)
        write_reg(SPI_CONTROL, soft_rst_n_reg, 0x1)
    else :
        write_reg(SPI_CONTROL, soft_rst_n_reg, 0x0)
        write_reg(SPI_CONTROL, chip_reg, chip_num)
        write_reg(SPI_CONTROL, cpol_reg, cpol)
        write_reg(SPI_CONTROL, cpha_reg, cpha)
        write_reg(SPI_CONTROL, w_r_mode_reg, 0x2)
        write_reg(SPI_CONTROL, wr_width_reg, rd_comd_width)
        write_reg(SPI_CONTROL, wr_data_reg, rd_command)
        write_reg(SPI_CONTROL, rd_width_reg, rd_data_width)
        write_reg(SPI_CONTROL, rd_target_num_reg, 0x1)
        write_reg(SPI_CONTROL, soft_rst_n_reg, 0x1)
    rd_done = read_reg(SPI_CONTROL, rd_done_reg)
    while rd_done!=1 :
        rd_done = read_reg(SPI_CONTROL, rd_done_reg)
    rx_data = read_reg(SPI_CONTROL, rd_data_reg)
    rx_data_bin = format(rx_data, '032b')[-rd_data_width:]
    rx_data = int(rx_data_bin, 2)
    print(f"Successfully read, the result is {rd_data_width}'b{rx_data_bin}.")
    return rx_data

In [ ]:
# MAX6675读取
rx_data = spi_read("MAX6675", 0x0)
temp = ((rx_data>>3)&0xFFF) * (1024/2**12)
print(f"Successfully read, the temperature is {temp}℃.")

In [ ]:
# BMI160读取
rd_flag = 1
rd_addr = 0x00
rx_data = spi_read("BMI160", (rd_flag<<7)|(rd_addr&0x7F))
print(hex(rx_data).upper())